### Data Analysis with Redshift Query Editor v2

Data Validation

In [0]:
SELECT 
   MIN(trading_date) as earliest_date,
   MAX(trading_date) as latest_date,
   COUNT(*) as total_rows,
   COUNT(DISTINCT trading_date) as unique_dates
FROM market_data.concentration_metrics;

Quality Assurance

In [0]:
SELECT 
    c.trading_date,
    c.mag7_pct_of_sp500,
    SUM(m.market_cap) as calc_mag7_cap,
    c.sp500_total_market_cap,
    (SUM(m.market_cap) / c.sp500_total_market_cap * 100) as calc_pct
FROM market_data.concentration_metrics c
JOIN market_data.magnificent7_metrics m 
    ON c.trading_date = m.trading_date
GROUP BY c.trading_date, c.mag7_pct_of_sp500, c.sp500_total_market_cap;

Time Series Analysis

In [0]:
SELECT 
    trading_date,
    total_mag7_market_cap,
    sp500_total_market_cap,
    mag7_pct_of_sp500,
    mag7_pct_of_sp500 - LAG(mag7_pct_of_sp500) 
        OVER (ORDER BY trading_date) as concentration_change
FROM market_data.concentration_metrics
ORDER BY trading_date;

In [0]:
Market Concentration Patterns

In [0]:
SELECT 
    m.trading_date,
    m.ticker,
    c.company_name,
    m.market_cap,
    m.pct_of_mag7,
    m.pct_of_sp500,
    RANK() OVER (PARTITION BY m.trading_date ORDER BY m.market_cap DESC) as size_rank
FROM market_data.magnificent7_metrics m
JOIN market_data.company_details c 
    ON m.trading_date = c.trading_date 
    AND m.ticker = c.ticker
WHERE m.trading_date = (SELECT MAX(trading_date) FROM market_data.magnificent7_metrics);

Query Performance Optimization

In [0]:
SELECT 
    trading_date,
    mag7_pct_of_sp500
FROM market_data.concentration_metrics
WHERE trading_date BETWEEN DATEADD(month, -3, GETDATE()) AND GETDATE()
ORDER BY trading_date;

In [0]:
SELECT 
    t.trading_date,
    t.ticker,
    t.close_price,
    m.market_cap,
    m.pct_of_sp500
FROM market_data.daily_trading t
JOIN market_data.magnificent7_metrics m 
    ON t.trading_date = m.trading_date 
    AND t.ticker = m.ticker
WHERE t.trading_date >= DATEADD(month, -1, GETDATE());